In [15]:
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
import sys
#!{sys.executable} -m  pip install --upgrade tensorflow
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional, MaxPooling1D, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform, he_uniform, zeros
from keras.optimizers import SGD, Adam, Adadelta
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras import metrics
from sklearn.externals import joblib

In [16]:
def import_textFile(filePath):
    ds_text = []
    with open(filePath,'rb') as f:
        for idx,ln in enumerate(f):
            decoded=False
            line=''
            for cp in ('cp1252', 'cp850','utf-8','utf8'):
                try:
                    line = ln.decode(cp)
                    decoded=True
                    break
                except UnicodeDecodeError:
                    pass
            if decoded:
                ds_text.insert(idx, line.rstrip())
    return ds_text

In [36]:
emo=5

In [38]:
if emo == 2: 
    dataset_path = '../Data/Datasets/Binary Classification/'
else:
    dataset_path = '../Data/Datasets/Multiclass Classification/'
    
train_ds = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test_ds = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)

x_train = train_ds[0]
y_train = train_ds[1]

x_test = test_ds[0]
y_test = test_ds[1]

In [39]:
maxLen = len(max(x_train, key=len).split())
print(maxLen)

23


In [40]:
def get_one_hot(y):
    one_hot_temp_array = array(y)
    one_hot_array = to_categorical(one_hot_temp_array)
    return one_hot_array
# invert encoding
#inverted = argmax(encoded[0])

In [41]:
#load glove diembedding_dict = dict()
embedding_dict = joblib.load('../Data/rnn_embedding_dict.pkl')
glove_words = joblib.load('../Data/rnn_glove_words.pkl')
number_to_word = glove_words
word_to_number = dict((word,idx) for idx,word in enumerate(glove_words))

In [42]:
def get_indices(input_x, word_to_number, max_len):
    m = input_x.shape[0]
    x_indices = np.zeros((m,max_len))

    for i in range(m):
        word_list =input_x.iloc[i][0].lower().split()
        for idx,word in enumerate(word_list):
            if (idx ==32):
                continue
            x_indices[i, idx] = word_to_number[word]
    return x_indices

In [43]:
def get_embedding_layer(embedding_dict, word_to_number):
    
    emb_shape = embedding_dict["cucumber"].shape[0]
    total_words = len(word_to_number) + 1
    
    emb_matrix = np.zeros((total_words,emb_shape))
    
    for word,idx in word_to_number.items():
        emb_matrix[idx, :] = embedding_dict[word]

    embedding_layer = Embedding(total_words, emb_shape, trainable = True)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [44]:
def senti_model(input_shape,embedding_dict, word_to_number):
    #np.random.seed(1)
    sentence_indices = Input(shape = input_shape, dtype = np.int32)
    embedding_layer = get_embedding_layer(embedding_dict, word_to_number)
    
    embeddings = embedding_layer(sentence_indices)   
    print(embeddings)
    X = Bidirectional(GRU(64,return_sequences=True))(embeddings)
    X = Dense(16, activation='elu')(X)
    X = Bidirectional(GRU(64,return_sequences=False))(X)
    X = Dense(64, activation='elu')(X)
    X = Dense(emo, activation='softmax', name='fc')(X)



    #X = Activation('softmax')(X)
    
    model = Model(inputs = sentence_indices, outputs = X, name='sentiment')
    
    return model

In [45]:
model = senti_model((maxLen,), embedding_dict, word_to_number)
model.summary()

Tensor("embedding_4/embedding_lookup/Identity:0", shape=(?, 23, 50), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 23)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 23, 50)            20000000  
_________________________________________________________________
bidirectional_7 (Bidirection (None, 23, 128)           44160     
_________________________________________________________________
dense_3 (Dense)              (None, 23, 16)            2064      
_________________________________________________________________
bidirectional_8 (Bidirection (None, 128)               31104     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
__________________________________________________________

In [46]:
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.0001)
adadelta = Adadelta(lr=1.00, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adadelta, metrics=[metrics.categorical_accuracy])

In [47]:
x_train_indices = get_indices(x_train, word_to_number, maxLen)
y_train_oh = get_one_hot(y_train)
print(y_train_oh.shape)

(2809, 5)


In [49]:
model.fit(x_train_indices, y_train_oh, epochs = 50, batch_size = 64, shuffle=True)

Epoch 1/50
2809/2809 [==============================] - 20s 7ms/step - loss: 1.5204 - categorical_accuracy: 0.2905
Epoch 2/50
2809/2809 [==============================] - 20s 7ms/step - loss: 1.5183 - categorical_accuracy: 0.2976
Epoch 3/50
2809/2809 [==============================] - 20s 7ms/step - loss: 1.5160 - categorical_accuracy: 0.2898
Epoch 4/50
2809/2809 [==============================] - 20s 7ms/step - loss: 1.5105 - categorical_accuracy: 0.3040
Epoch 5/50
2809/2809 [==============================] - 20s 7ms/step - loss: 1.5093 - categorical_accuracy: 0.3015
Epoch 6/50
2809/2809 [==============================] - 21s 8ms/step - loss: 1.5103 - categorical_accuracy: 0.3054
Epoch 7/50
2809/2809 [==============================] - 20s 7ms/step - loss: 1.5095 - categorical_accuracy: 0.3033
Epoch 8/50
2809/2809 [==============================] - 20s 7ms/step - loss: 1.5059 - categorical_accuracy: 0.3133
Epoch 9/50
2809/2809 [==============================] - 21s 7ms/step - loss: 1.5

In [ ]:
x_test_indices = get_indices(x_test, word_to_number, max_len = maxLen)
y_test_oh = get_one_hot(y_test)
loss, acc = model.evaluate(x_test_indices, y_test_oh)
print()
print("Test accuracy = ", acc)

In [ ]:
def get_emotion(value):
    unique_labels = ["Happy","Surprise","Anger","Fear","Sadness"]
    return unique_labels[value]

In [ ]:
C = 4
y_test_array = y_test.values
y_test_oh = np.eye(C)[y_test_array.reshape(-1)]
X_test_indices = get_indices(x_test, word_to_number, maxLen)
pred = model.predict(x_test_indices)
for i in range(len(x_test)):
    x = x_test_indices
    num = np.argmax(pred[i])
    if(num != y_test_array[i]):
        print('Expected '+ get_emotion(y_test_array[i]) + ' prediction: '+ get_emotion(num) + ' ' + x_test[i] )